# Experiment 1 Baseline

Checking whether an array is a heap using an RNN

In [1]:
%cd ..
%pwd
import wandb
from experiments.heaps import *
from experiments.utils import *
from baselines.baseline_rnn import *
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

/home/tekne/Oxford/ATML/Project/ggs-nn-model
